<a href="https://colab.research.google.com/github/zvyagind/parsing/blob/2_Lesson/%D0%A3%D1%80%D0%BE%D0%BA_2_%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_HTML_BeautifulSoup%2C_MongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Вариант 1
Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
    Наименование вакансии.
    Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
    Ссылку на саму вакансию.
    Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

## imports

In [2]:
import requests
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from bs4 import BeautifulSoup as bs
from pprint import pprint

from random import randint
from time import sleep


## set starting params

In [3]:
url = 'https://hh.ru/search/vacancy?searchVacancy&text=' #какая-то беда с кодировкой при передаче кириллицы в requests без полной строки url, поэтому не со стартовой страницы

headers = {'User-Agent':'Mozilla/5.0 (Linux; Android 10; VOG-L29) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Mobile Safari/537.36 OPR/63.3.3216.58675',
           'Accept':	'text/html',
           'Referer':	'http://www.google.com/'}

#####   INPUT   #####

query = 'аналитик+данных' #'+'.join(input('Введите интересующую вакансию: ').split(' ')).lower()
lim = 5 #int(input('Введите кол-во страниц для поиска (по 20 вакансий на страницу): '))

#####   INPUT   #####

# определение параметров для поиска bs в зависимости от полей, которые хотим получить (для разных сайтов делаем ещё один ключ)
params = {'title': ('a', {'class': 'bloko-link', 'data-qa':'vacancy-serp__vacancy-title'}),
          'link': ('a', {'class': 'bloko-link', 'data-qa':'vacancy-serp__vacancy-title'}),
          'employer': ('a', {'class': 'bloko-link bloko-link_secondary', 'data-qa':'vacancy-serp__vacancy-employer'}),
          'city': ('span', {'class': "vacancy-serp-item__meta-info", 'data-qa':"vacancy-serp__vacancy-address"}),
          'wage':('span', {'data-qa':"vacancy-serp__vacancy-compensation"})}


## def get_next_page

In [4]:
def get_next_page(soup):
    try:
        next_url = 'https://hh.ru'+soup.find('a', {'class':'bloko-button', 'data-qa': 'pager-next'})['href']
    except:
        next_url = None
    # print(next_url)
    return next_url


## def get_vac_data

In [5]:
next_url = url+query # starting point

def get_vac_data(next_url, headers, lim):
    data = pd.DataFrame(columns = ['title', 'link', 'employer', 'city', 'wage']) # , 'wagefrom', 'wageto', 'currency'
    p = 0
    
    while True:
        if next_url == None or p == lim+1:
            print('Остановка - нет следующего url или достигнут лимит страниц')
            return data

        sleep(randint(1,5))
        response = requests.get(next_url, headers=headers)
        soup = bs(response.text, 'html.parser')
        vacancies = soup.find_all('div', {'class':'vacancy-serp-item'})
        
        for item in vacancies:
            data_dict = {}
            for i in params.keys():
                try:
                    data_dict[i] = item.find(params[i][0], params[i][1]).getText().replace('\u202f','').replace('\xa0',' ')
                    if i == 'link':
                        data_dict[i] = item.find(params[i][0], params[i][1]).get('href').split('?')[0]
                    if i == 'city':
                        data_dict[i] = item.find(params[i][0], params[i][1]).getText().replace('\u202f','').replace('\xa0',' ').split(',')[0]
                except:
                    data_dict[i] = None
            
            data = data.append(data_dict, ignore_index = True)
            # print(data.to_markdown())
        
        next_url = get_next_page(soup)
        p +=1 # счетчик страниц
    
    return data


## execution

In [6]:
data = get_vac_data(next_url, headers, lim)
# print(data.head(100).to_markdown())

Остановка - нет следующего url или достигнут лимит страниц


## transform

In [7]:
data['wage_from'] = 0
data['wage_to'] = 0
data['wage_currency'] = 'руб.'

for i in range(len(data['wage'])):
    if data['wage'][i] == None:
        data['wage_from'][i] = None
        data['wage_to'][i] = None
        data['wage_currency'][i] = None
        continue
    if len([int(s) for s in str.split(data['wage'][i]) if s.isdigit()]) == 1:
        if data['wage'][i].split(' ')[0] == 'от':
            data['wage_from'][i] = int(data['wage'][i].split(' ')[1])
        else:
            data['wage_to'][i] = int(data['wage'][i].split(' ')[1])
    else:
        data['wage_from'][i] = int(data['wage'][i].split(' ')[0])
        data['wage_to'][i] = int(data['wage'][i].split(' ')[2])
    data['wage_currency'][i] = data['wage'][i].split('0 ')[-1]

print(data.head(40).to_markdown())

|    | title                                                                               | link                           | employer                                           | city            | wage                 |   wage_from |   wage_to | wage_currency   |
|---:|:------------------------------------------------------------------------------------|:-------------------------------|:---------------------------------------------------|:----------------|:---------------------|------------:|----------:|:----------------|
|  0 | Аналитик                                                                            | https://hh.ru/vacancy/47218683 | Фабрика Кофе                                       | Москва          | 50000 – 70000 руб.   |       50000 |     70000 | руб.            |
|  1 | Начинающий специалист со знанием Excel (аналитик)                                   | https://hh.ru/vacancy/46914727 | "Рельеф-Центр" Департамент Административный блок   | Рязань          | от 30000 ру

## save

In [8]:
data.to_csv(u'hh_request.csv', sep = ';', encoding='utf-16')

# Вариант 2
Необходимо собрать информацию по продуктам питания с сайта: Список протестированных продуктов на сайте Росконтроль.рф Приложение должно анализировать несколько страниц сайта (вводим через input или аргументы).
Получившийся список должен содержать:

Наименование продукта.
Все параметры (Безопасность, Натуральность, Пищевая ценность, Качество) Не забываем преобразовать к цифрам.
Общую оценку
Сайт, откуда получена информация.
Общий результат можно вывести с помощью dataFrame через Pandas. Сохраните в json либо csv.

## set starting params

In [59]:
url = 'https://roscontrol.com'
products_url = 'https://roscontrol.com/category/produkti/'

headers = {'User-Agent':'Mozilla/5.0 (Linux; Android 10; VOG-L29) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Mobile Safari/537.36 OPR/63.3.3216.58675',
           'Accept':	'text/html',
           'Referer':	'http://www.google.com/'}


## get categories list with urls

In [60]:
products = requests.get(products_url, headers=headers)
soup = bs(products.text, 'html.parser')
prod = soup.find_all('div', {'class':'grid-padding grid-column-3 grid-column-large-6 grid-flex-mobile grid-column-middle-6 grid-column-small-12 grid-left'})
prod_class_list = []
for i in prod:
    prod_class = {}
    prod_class['name'] = i.find('div',{'class': 'catalog__category-name'}).text
    prod_class['link'] = i.find('a',{'class': 'catalog__category-item util-hover-shadow'})['href']
    prod_class_list.append(prod_class)


In [61]:
prod_type_list = []
for j in prod_class_list:
    sleep(randint(1, 5))
    products = requests.get(url+j['link'], headers=headers)
    soup = bs(products.text, 'html.parser')
    prod = soup.find_all('div', {'class':'grid-padding grid-column-3 grid-column-large-6 grid-flex-mobile grid-column-middle-6 grid-column-small-12 grid-left'})
    for i in prod:
        prod_type = {}
        prod_type['class']=j['name']
        prod_type['type'] = i.find('div',{'class': 'catalog__category-name'}).text
        prod_type['link'] = i.find('a',{'class': 'catalog__category-item util-hover-shadow'})['href']
        prod_type_list.append(prod_type)


## get data

In [58]:
lim = 2 # Макс глубина сканирования

prod_list = []
for i in prod_type_list:

    for j in range (1, lim+1):
        
        # if i['class']!=' Молочные продукты ':
        #     continue

        sleep(randint(1, 3))
        products = requests.get(url+i['link']+f'?page={j}', headers=headers)

        #####print(url+i['link']+f'?page={j}')

        soup = bs(products.text, 'html.parser')
        if j > 1 and soup.find('span', {'class':'hide-with-1024'}) == None:

            #####print('Пропущена - ', url+i['link']+f'?page={j}')
            continue
        prod = soup.find_all('div', {'class':'product__item-leftblock group'})
        
        for k in prod:
            prod_type = {}
            prod_type['class']=i['class']
            prod_type['type'] = i['type']
            prod_type['link'] = url+i['link']+f'?page={j}'

            prod_type['name'] = k.find('div', {'class': 'product__item-link'}).text

            quality = k.find_all('span', {'class': 'rating_row'})
            try:
                prod_type['safety'] = int(quality[0].find('i')['data-width'])
            except:
                prod_type['safety'] = None
            try:
                prod_type['origin'] = int(quality[1].find('i')['data-width'])
            except:
                prod_type['origin'] = None
            try:
                prod_type['nutritional value'] = int(quality[2].find('i')['data-width'])
            except:
                prod_type['nutritional value'] = None
            try:
                prod_type['quality'] = int(quality[3].find('i')['data-width'])
            except:
                prod_type['quality'] = None

            try:
                prod_type['total score'] = int(k.find('div', {'class': 'rating-value'}).text)
            except:
                prod_type['total score'] = None

            prod_list.append(prod_type)
df = pd.DataFrame(prod_list)

In [62]:
df.tail(20)

,class,type,link,name,safety,origin,nutritional value,quality,total score
3075,"Фрукты, овощи, грибы",Картофель,https://roscontrol.com/category/produkti/ovosh...,"Картофель ""Перекрёсток"" мытый",60.0,90.0,NaN,NaN,75.0
3076,"Фрукты, овощи, грибы",Картофель,https://roscontrol.com/category/produkti/ovosh...,"Картофель ""Лента"" мытый",51.0,84.0,NaN,NaN,68.0
3077,"Фрукты, овощи, грибы",Картофель,https://roscontrol.com/category/produkti/ovosh...,"Картофель ""Ашан""",46.0,19.0,NaN,NaN,NaN
3078,"Фрукты, овощи, грибы",Картофель,https://roscontrol.com/category/produkti/ovosh...,"Картофель ""Магнит""",78.0,42.0,NaN,NaN,NaN
3079,"Фрукты, овощи, грибы",Картофель,https://roscontrol.com/category/produkti/ovosh...,"Картофель ""Дикси""",56.0,23.0,NaN,NaN,NaN
3080,"Фрукты, овощи, грибы",Картофель,https://roscontrol.com/category/produkti/ovosh...,"Картофель ""Овощная лига"" для варки",78.0,27.0,NaN,NaN,NaN
3081,"Фрукты, овощи, грибы",Картофель,https://roscontrol.com/category/produkti/ovosh...,"Картофель ""Вкусные овощи"" для варки",76.0,40.0,NaN,NaN,NaN
3082,"Фрукты, овощи, грибы",Картофель,https://roscontrol.com/category/produkti/ovosh...,"Картофель ""Агроторг"" мытый",79.0,35.0,NaN,NaN,NaN
3083,"Фрукты, овощи, грибы",Картофель,https://roscontrol.com/category/produkti/ovosh...,Картофель Galey Moshe для варки,79.0,36.0,NaN,NaN,NaN
3084,"Фрукты, овощи, грибы",Картофель,https://roscontrol.com/category/produkti/ovosh...,"Картофель ""Карусель"" белый мытый",60.0,41.0,NaN,NaN,NaN


## save

In [63]:
df.to_csv(u'roscontrol.csv', sep = ';', encoding='utf-16')